In [1]:
"""
Tarea opcional: Parcial 3

Computacion estadistica
UNAL 2023-1

Willian Andrés Cárdenas Sandoval
"""

'\nTarea opcional: Parcial 3\n\nComputacion estadistica\nUNAL 2023-1\n\nWillian Andrés Cárdenas Sandoval\n'

In [2]:
import os 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mismacrospython import onewayfreq, twowayfreq
!pip install duckdb
!pip install openpyxl 
import duckdb
con = duckdb.connect(database=':memory:')

pd.options.display.float_format = '{:,.2f}'.format


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
games = pd.read_table("games.csv",delimiter=",",header=0,
                      usecols=["gameID","leagueID","season","homeTeamID","awayTeamID","homeGoals","awayGoals"]
                      )
teams = pd.read_excel("UEFA5.xlsx",sheet_name="teams",header=0)
leagues = pd.read_excel("UEFA5.xlsx",sheet_name="leagues",header=0)
players = pd.read_excel("UEFA5.xlsx",sheet_name="players",header=0)
appearances = pd.read_table("appearances.csv",delimiter=",",header=0,quotechar="'",usecols=[0,1,2,15],
                            names=["gameID","playerID","goals","time"]
                           )
appearances["playerID"] = pd.to_numeric(appearances["playerID"].str.replace('"',"",regex=True))
appearances["gameID"] = pd.to_numeric(appearances["gameID"].str.replace('"',"",regex=True))
games = con.execute("""select   games.*, leagues.leagueNAME
                     from     games left join leagues on (games.leagueID = leagues.leagueID)
                     """).df()
appearances = con.execute("""select   appearances.*, players.playerNAME
                     from     appearances left join players on (appearances.playerID = players.playerID)
                     """).df()

In [4]:
#Tabla de una temporada de una liga de futbol
def tabla_liga(season,liga,posicion = 0):
    #Liga y season
    p1 = games[(games['season'] == season) & (games['leagueNAME'] == liga)]
    #Equipos de la liga id Y Nombres
    res = pd.DataFrame(columns=['EquipoId', 'EquipoName', 'Victorias', 'Derrotas', 'Empates', 'GF', 'GC', 'DF'])
    res["EquipoId"] = p1["awayTeamID"].unique()
    merged_df = pd.merge(res, teams, left_on="EquipoId", right_on="teamID", how="left")
    res["EquipoName"] = merged_df["teamNAME"]
    #Equipo Victorias, derrotas, empates, goles a favor y goles en contra
    victoriasA = []
    empatesA = []
    derrotasA = []
    gfA = []
    gcA = []


    for equipo_id in res["EquipoId"]:
        #Filtrar los juegos del equipo en el DataFrame games
        juegos_equipo = p1[p1["awayTeamID"] == equipo_id]
        #Contar las victorias, empates y derrotas del equipo
        victorias_equipo = len(juegos_equipo[juegos_equipo["awayGoals"] > juegos_equipo["homeGoals"]])
        empates_equipo = len(juegos_equipo[juegos_equipo["awayGoals"] == juegos_equipo["homeGoals"]])
        derrotas_equipo = len(juegos_equipo[juegos_equipo["awayGoals"] < juegos_equipo["homeGoals"]])
        gf_equipo = juegos_equipo["awayGoals"].sum()
        gc_equipo = juegos_equipo["homeGoals"].sum()

        #Resultados
        victoriasA.append(victorias_equipo)
        empatesA.append(empates_equipo)
        derrotasA.append(derrotas_equipo)
        gfA.append(gf_equipo)
        gcA.append(gc_equipo)

    victoriasH = []
    empatesH = []
    derrotasH = []
    gfH = []
    gcH = []

    for equipo_id in res["EquipoId"]:
        #Filtrar los juegos del equipo en el DataFrame games
        juegos_equipo = p1[p1["homeTeamID"] == equipo_id]
        #Contar las victorias, empates y derrotas del equipo
        victorias_equipo = len(juegos_equipo[juegos_equipo["homeGoals"] > juegos_equipo["awayGoals"]])
        empates_equipo = len(juegos_equipo[juegos_equipo["homeGoals"] == juegos_equipo["awayGoals"]])
        derrotas_equipo = len(juegos_equipo[juegos_equipo["homeGoals"] < juegos_equipo["awayGoals"]])
        gf_equipo = juegos_equipo["homeGoals"].sum()
        gc_equipo = juegos_equipo["awayGoals"].sum()

        #Resultados
        victoriasH.append(victorias_equipo)
        empatesH.append(empates_equipo)
        derrotasH.append(derrotas_equipo)
        gfH.append(gf_equipo)
        gcH.append(gc_equipo)

    res["Victorias"] = np.array(victoriasA) + np.array(victoriasH) 
    res["Empates"] = np.array(empatesA) + np.array(empatesH)
    res["Derrotas"] = np.array(derrotasA) + np.array(derrotasH)
    res["GF"] = np.array(gfH) + np.array(gfA)
    res["GC"] = np.array(gcH) + np.array(gcA)

    #Puntos y diferencia de gol
    res["Puntos"] = (3 * res["Victorias"]) + res["Empates"]
    res["DF"] = res["GF"] - res["GC"]
    res = res.sort_values(by=["Puntos", "DF", "GF"], ascending=False)
    if posicion == 0:
        display(res)
    else:
        display(res.iloc[posicion - 1])
    return res

In [5]:
#Punto 1
tabla_liga(2016,"Premier League",1)
# Chelsea (80) | Puntos 93 | Diferencia de gol 52

EquipoId           80
EquipoName    Chelsea
Victorias          30
Derrotas            5
Empates             3
GF                 85
GC                 33
DF                 52
Puntos             93
Name: 14, dtype: object

,EquipoId,EquipoName,Victorias,Derrotas,Empates,GF,GC,DF,Puntos
14,80,Chelsea,30,5,3,85,33,52,93
2,82,Tottenham,26,4,8,86,26,60,86
11,88,Manchester City,23,6,9,80,39,41,78
8,87,Liverpool,22,6,10,78,42,36,76
16,83,Arsenal,23,9,6,77,44,33,75
7,89,Manchester United,18,5,15,54,29,25,69
15,72,Everton,17,11,10,62,44,18,61
10,74,Southampton,12,16,10,41,48,-7,46
18,73,Bournemouth,12,16,10,55,67,-12,46
1,76,West Bromwich Albion,12,17,9,43,51,-8,45


In [6]:
#Punto 2
tabla_liga(2017,"Serie A",2)
# Juventus (105) | Puntos 91 | Diferencia de gol 48

EquipoId           105
EquipoName    Juventus
Victorias           28
Derrotas             3
Empates              7
GF                  77
GC                  29
DF                  48
Puntos              91
Name: 1, dtype: object

,EquipoId,EquipoName,Victorias,Derrotas,Empates,GF,GC,DF,Puntos
10,98,Napoli,30,3,5,86,24,62,95
1,105,Juventus,28,3,7,77,29,48,91
2,95,Roma,23,7,8,61,28,33,77
16,96,Lazio,21,8,9,89,49,40,72
11,106,Inter,20,6,12,66,30,36,72
9,111,AC Milan,18,10,10,56,42,14,64
14,107,Atalanta,16,10,12,57,39,18,60
4,110,Fiorentina,16,13,9,54,46,8,57
3,113,Torino,13,10,15,54,46,8,54
18,102,Sampdoria,16,16,6,56,60,-4,54


In [7]:
#Punto 3
tabla_liga(2018,"Bundesliga",3)
# Bayern Munich (136) | Puntos 66 | Diferencia de gol 34

EquipoId                136
EquipoName    Bayern Munich
Victorias                19
Derrotas                  6
Empates                   9
GF                       63
GC                       29
DF                       34
Puntos                   66
Name: 8, dtype: object

,EquipoId,EquipoName,Victorias,Derrotas,Empates,GF,GC,DF,Puntos
15,117,RasenBallsport Leipzig,24,4,6,88,32,56,78
9,129,Borussia Dortmund,23,4,7,81,44,37,76
8,136,Bayern Munich,19,6,9,63,29,34,66
6,119,Bayer Leverkusen,18,12,4,69,52,17,58
12,130,Borussia M.Gladbach,16,11,7,55,42,13,55
14,131,Wolfsburg,16,11,7,62,50,12,55
5,132,Eintracht Frankfurt,15,10,9,60,48,12,54
11,123,Werder Bremen,14,9,11,58,49,9,53
0,120,Hoffenheim,13,9,12,70,52,18,51
16,233,Fortuna Duesseldorf,13,16,5,49,65,-16,44


In [8]:
#Punto 4
tabla_liga(2019,"La Liga",4)
# Real Madrid (138) | Puntos 70 | Diferencia de gol 20

EquipoId              138
EquipoName    Real Madrid
Victorias              19
Derrotas                6
Empates                13
GF                     54
GC                     34
DF                     20
Puntos                 70
Name: 7, dtype: object

,EquipoId,EquipoName,Victorias,Derrotas,Empates,GF,GC,DF,Puntos
0,150,Sevilla,26,3,9,70,25,45,87
3,148,Barcelona,25,6,7,86,38,48,82
10,143,Atletico Madrid,18,4,16,51,27,24,70
7,138,Real Madrid,19,6,13,54,34,20,70
14,154,Villarreal,18,14,6,63,49,14,60
1,140,Real Sociedad,16,14,8,56,48,8,56
5,155,Granada,16,14,8,52,45,7,56
9,142,Getafe,14,12,12,43,37,6,54
11,146,Valencia,14,13,11,46,53,-7,53
4,157,Osasuna,13,12,13,46,54,-8,52


In [9]:
#Punto 5
tabla_liga(2020,"Ligue 1",5)
# Lille (164) | Puntos 60 | Diferencia de gol 7

EquipoId        164
EquipoName    Lille
Victorias        16
Derrotas         10
Empates          12
GF               54
GC               47
DF                7
Puntos           60
Name: 8, dtype: object

,EquipoId,EquipoName,Victorias,Derrotas,Empates,GF,GC,DF,Puntos
13,160,Marseille,24,3,11,64,23,41,83
9,161,Paris Saint Germain,26,8,4,86,28,58,82
11,171,Monaco,24,8,6,76,42,34,78
17,178,Lyon,22,6,10,81,43,38,76
8,164,Lille,16,10,12,54,47,7,60
2,163,Rennes,16,12,10,52,40,12,58
5,210,Lens,15,11,12,55,54,1,57
14,166,Montpellier,14,12,12,60,62,-2,54
16,170,Nice,15,16,7,50,53,-3,52
18,180,Metz,12,15,11,44,48,-4,47


In [10]:
#Tabla de jugadores de una liga y una temporada con más de 10 goles
def jugadores_top(liga,season):
    #Liga y season
    p2 = games[(games['season'] == season) & (games['leagueNAME'] == liga)]
    #Resultado
    base = pd.merge(appearances, p2, left_on="gameID", right_on="gameID", how="inner")
    res = pd.DataFrame(columns=['JugadorId', 'JugadorName','Goles', 'Tiempo','Promedio'])
    res["JugadorId"] = base.groupby("playerID")["playerID"].first().values
    res["JugadorName"] = base.groupby("playerID")["playerNAME"].first().values
    res["Goles"] = base.groupby("playerID")["goals"].sum().values
    res["Tiempo"] = base.groupby("playerID")["time"].sum().values
    res["Promedio"] = 90 * (res["Goles"]/res["Tiempo"])

    res = res.query("Goles > 10").sort_values("Promedio", ascending=False)
    display(res.head(1))
    return res 

In [11]:
#Punto 6
jugadores_top("Ligue 1",2016)
# Edinson Cavani (3294) | 1.055983909

,JugadorId,JugadorName,Goles,Tiempo,Promedio
101,3294,Edinson Cavani,35,2983,1.06


,JugadorId,JugadorName,Goles,Tiempo,Promedio
101,3294,Edinson Cavani,35,2983,1.06
90,3277,Alexandre Lacazette,28,2416,1.04
9,696,Falcao,21,1969,0.96
186,3423,Kylian Mbappe-Lottin,15,1503,0.90
24,1817,Mario Balotelli,15,1748,0.77
10,717,Bafétimbi Gomis,20,2613,0.69
70,3244,Nicolas de Preville,14,2077,0.61
281,3570,Benjamin Moukandjo,13,2017,0.58
116,3310,Alassane Pléa,11,1853,0.53
484,5701,Ivan Santini,15,2905,0.46


In [12]:
#Punto 7
jugadores_top("La Liga",2017)
# Lionel Messi (2097) | 1.021702838

,JugadorId,JugadorName,Goles,Tiempo,Promedio
89,2097,Lionel Messi,34,2995,1.02


,JugadorId,JugadorName,Goles,Tiempo,Promedio
89,2097,Lionel Messi,34,2995,1.02
249,2371,Cristiano Ronaldo,26,2304,1.02
164,2251,Gareth Bale,16,1803,0.80
90,2098,Luis Suárez,25,2902,0.78
257,2384,Santi Mina,12,1548,0.70
53,1709,Cristhian Stuani,21,2728,0.69
181,2270,Antoine Griezmann,19,2532,0.68
199,2290,Iago Aspas,22,2939,0.67
47,1642,Simone Zaza,13,2082,0.56
32,1125,Carlos Bacca,15,2513,0.54


In [13]:
#Punto 8
jugadores_top("Bundesliga",2018)
# Paco Alcácer (2380) | 1.371718882

,JugadorId,JugadorName,Goles,Tiempo,Promedio
207,2380,Paco Alcácer,18,1181,1.37


,JugadorId,JugadorName,Goles,Tiempo,Promedio
207,2380,Paco Alcácer,18,1181,1.37
243,3834,Ishak Belfodil,16,1867,0.77
349,6228,Luka Jovic,17,2256,0.68
103,227,Robert Lewandowski,22,2959,0.67
144,319,Marco Reus,17,2335,0.66
192,956,Andrej Kramaric,17,2418,0.63
345,6144,Sébastien Haller,15,2176,0.62
27,65,Timo Werner,16,2537,0.57
284,5250,Yussuf Poulsen,15,2450,0.55
266,5220,Kai Havertz,17,2906,0.53


In [14]:
#Punto 9
jugadores_top("Serie A",2019)
# Luis Muriel (1229) | 1.32244898

,JugadorId,JugadorName,Goles,Tiempo,Promedio
90,1229,Luis Muriel,18,1225,1.32


,JugadorId,JugadorName,Goles,Tiempo,Promedio
90,1229,Luis Muriel,18,1225,1.32
81,1209,Ciro Immobile,36,3179,1.02
292,2371,Cristiano Ronaldo,31,2920,0.96
158,1426,Josip Ilicic,15,1682,0.80
61,1148,Duván Zapata,18,2047,0.79
354,6156,Andreas Cornelius,12,1482,0.73
262,1852,Ante Rebic,11,1417,0.70
17,594,Romelu Lukaku,23,2985,0.69
411,6980,Francesco Caputo,21,2940,0.64
230,1583,Arkadiusz Milik,11,1754,0.56


In [15]:
#Punto 10
jugadores_top("Premier League",2020)
# Premier League (2251) | 1.089108911

,JugadorId,JugadorName,Goles,Tiempo,Promedio
215,2251,Gareth Bale,11,909,1.09


,JugadorId,JugadorName,Goles,Tiempo,Promedio
215,2251,Gareth Bale,11,909,1.09
66,620,Kelechi Iheanacho,12,1452,0.74
76,647,Harry Kane,23,3097,0.67
168,1250,Mohamed Salah,22,3085,0.64
235,3277,Alexandre Lacazette,13,1939,0.60
10,314,Ilkay Gündogan,13,2033,0.58
165,1228,Bruno Fernandes,18,3117,0.52
19,468,Callum Wilson,12,2079,0.52
276,5555,Dominic Calvert-Lewin,16,2878,0.50
125,822,Patrick Bamford,17,3085,0.50


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd2fca3f-dc64-4c71-8bd4-adda531fa911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>